In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry 

from collections import namedtuple

%matplotlib inline
pd.set_option('display.float_format', lambda x : '{:.2f}'.format(x))

In [37]:
df_roam = pd.read_csv('../input/corona/Roaming_data.csv')
df_infect = pd.read_excel('../input/corona/Global_infection.xls')

In [38]:
df_infect.head()

,filename,title,URL,CollectionDate,PostDate,Channel,Influenza,Class
0,AFRO2019120200003,WHO supports Angola’s Government efforts to en...,https://www.afro.who.int/news/who-supports-ang...,2019-12-02 21:00:16.0,2019-12-02 00:00:00,AFRO,Poliomyelitis,Viral
1,AFRO2019120300001,African First Ladies laud progress against HIV...,https://www.afro.who.int/news/african-first-la...,2019-12-03 00:00:17.0,2019-12-02 00:00:00,AFRO,Acquired immuno deficiency syndrome,Viral
2,AFRO2019120300002,Eritrea Observed World AIDS Day 2019,https://www.afro.who.int/news/eritrea-observed...,2019-12-03 00:00:18.0,2019-12-02 00:00:00,AFRO,Acquired immuno deficiency syndrome,Viral
3,AFRO2019120400003,West and Central Africa Expands Access to HIV ...,https://www.afro.who.int/news/west-and-central...,2019-12-04 03:00:14.0,2019-12-03 00:00:00,AFRO,Acquired immuno deficiency syndrome,Viral
4,AFRO2019120400004,Nearly 23 million voluntary male medical circu...,https://www.afro.who.int/news/nearly-23-millio...,2019-12-04 22:00:14.0,2019-12-04 00:00:00,AFRO,Acquired immuno deficiency syndrome,Viral


In [39]:
df_roam.head()


,return,iso,arrival,departure,count
0,20200131,ad,20200130,20200130,3
1,20200131,ae,20191231,20200125,3
2,20200131,ae,20191231,20200131,3
3,20200131,ae,20200102,20200102,3
4,20200131,ae,20200104,20200131,3


In [40]:
df_infect.drop(['filename', 'title', 'URL', 'CollectionDate', 'Channel', 'Class'], axis= 1, inplace=True)

In [41]:
df_infect.head()

,PostDate,Influenza
0,2019-12-02 00:00:00,Poliomyelitis
1,2019-12-02 00:00:00,Acquired immuno deficiency syndrome
2,2019-12-02 00:00:00,Acquired immuno deficiency syndrome
3,2019-12-03 00:00:00,Acquired immuno deficiency syndrome
4,2019-12-04 00:00:00,Acquired immuno deficiency syndrome


In [42]:
print('Influenza Count\n',df_infect.Influenza.value_counts(),'\n')
print('Influenza Percentage\n',df_infect.Influenza.value_counts('.'))

Influenza Count
 COVID-19                             35238
Influenza                             6760
Dengue fever                          3945
Measles                               1587
Severe acute respiratory syndrome     1574
                                     ...  
Hymenolepiasis                           1
Fowl tuberculosis                        1
Rhinovirus infection                     1
Paragonimiasis                           1
Onchocercosis                            1
Name: Influenza, Length: 219, dtype: int64 

Influenza Percentage
 COVID-19                            0.57
Influenza                           0.11
Dengue fever                        0.06
Measles                             0.03
Severe acute respiratory syndrome   0.03
                                    ... 
Hymenolepiasis                      0.00
Fowl tuberculosis                   0.00
Rhinovirus infection                0.00
Paragonimiasis                      0.00
Onchocercosis                    

In [43]:
counts = df_infect.Influenza.value_counts()
Upcounts = counts[counts>100]
Upcounts

COVID-19                                             35238
Influenza                                             6760
Dengue fever                                          3945
Measles                                               1587
Severe acute respiratory syndrome                     1574
Poliomyelitis                                         1167
Acquired immuno deficiency syndrome                   1127
Ebola                                                  944
Tuberculosis                                           876
Avian influenza                                        825
Malaria                                                557
Middle East respiratory syndrome                       464
Rabies                                                 384
Lassa fever                                            382
measles                                                339
Plague                                                 304
Noroviral gastroenteritis                              2

In [44]:
print('전체 index 갯수:',len(counts))
print('뽑아 낸 유용한 index 갯수:',len(Upcounts))

전체 index 갯수: 219
뽑아 낸 유용한 index 갯수: 29


In [45]:
temp = df_infect.Influenza.isin(Upcounts.index)
df_infect = df_infect[temp]

In [46]:
df_infect.Influenza.value_counts()

COVID-19                                             35238
Influenza                                             6760
Dengue fever                                          3945
Measles                                               1587
Severe acute respiratory syndrome                     1574
Poliomyelitis                                         1167
Acquired immuno deficiency syndrome                   1127
Ebola                                                  944
Tuberculosis                                           876
Avian influenza                                        825
Malaria                                                557
Middle East respiratory syndrome                       464
Rabies                                                 384
Lassa fever                                            382
measles                                                339
Plague                                                 304
Noroviral gastroenteritis                              2

In [47]:
f, ax = plt.subplots(figsize=(150, 100))
sns.countplot('Influenza', data=df_infect)
plt.title('Influneza Values Counts', fontsize=100)
plt.xticks(rotation='80', fontsize = 100)
plt.yticks(rotation='70', fontsize = 100)
plt.xlabel('Influenza Values', fontsize=100)
plt.ylabel('Counts', fontsize=100 )
plt.tight_layout()

In [48]:
df_infect.PostDate.value_counts()

2020-03-31 00:00:00    687
2020-03-30 00:00:00    680
2020-03-25 00:00:00    654
2020-03-27 00:00:00    648
2020-03-26 00:00:00    624
                      ... 
2020-01-30 00:01:00      1
2020-02-28 01:59:00      1
2020-01-24 13:44:00      1
2020-01-12 18:45:03      1
2020-03-12 05:42:00      1
Name: PostDate, Length: 25396, dtype: int64

In [49]:
test = df_infect.PostDate.value_counts()
uTest = test[test>100]
len(uTest)

88

In [50]:
temp1 = df_infect.PostDate.isin(uTest.index)
df_infect = df_infect[temp1]

In [51]:
df_infect.PostDate.value_counts()

2020-03-31 00:00:00    687
2020-03-30 00:00:00    680
2020-03-25 00:00:00    654
2020-03-27 00:00:00    648
2020-03-26 00:00:00    624
                      ... 
2019-12-09 00:00:00    110
2020-02-23 00:00:00    109
2019-12-18 00:00:00    108
2020-03-14 00:00:00    107
2019-12-17 00:00:00    103
Name: PostDate, Length: 88, dtype: int64

In [52]:
"""
f, ax = plt.subplots(figsize=(150, 100))
sns.countplot('PostDate', data=df_infect)
plt.title('PostDate Values Counts', fontsize=100)
plt.xticks(rotation='90', fontsize = 70)
plt.yticks(rotation='70', fontsize = 100)
plt.xlabel('PostDate Values', fontsize=100)
plt.ylabel('Counts', fontsize=100 )
plt.tight_layout()
"""

"\nf, ax = plt.subplots(figsize=(150, 100))\nsns.countplot('PostDate', data=df_infect)\nplt.title('PostDate Values Counts', fontsize=100)\nplt.xticks(rotation='90', fontsize = 70)\nplt.yticks(rotation='70', fontsize = 100)\nplt.xlabel('PostDate Values', fontsize=100)\nplt.ylabel('Counts', fontsize=100 )\nplt.tight_layout()\n"

In [53]:
df_infect.PostDate.sort_values

<bound method Series.sort_values of 0        2019-12-02 00:00:00
1        2019-12-02 00:00:00
2        2019-12-02 00:00:00
3        2019-12-03 00:00:00
4        2019-12-04 00:00:00
                ...         
61717    2020-02-16 00:00:00
61718    2020-02-16 00:00:00
61719    2020-03-16 00:00:00
61720    2020-03-16 00:00:00
61721    2020-03-16 00:00:00
Name: PostDate, Length: 22470, dtype: object>

In [54]:
df_infect['Date'] = pd.to_datetime(df_infect['PostDate'])
df_infect.drop('PostDate', axis=1, inplace=True)

In [55]:
df_infect.head()
df_infect.tail()

,Influenza,Date
61717,Influenza,2020-02-16
61718,Influenza,2020-02-16
61719,Influenza,2020-03-16
61720,Influenza,2020-03-16
61721,Influenza,2020-03-16


In [56]:
df_infect.sort_values(by='Date', ascending=True)
f, ax = plt.subplots(figsize=(150, 100))
sns.countplot('Date', data=df_infect)
plt.title('x: PostDate & y: Counts', fontsize=100)
plt.xticks(rotation='90', fontsize = 70)
plt.yticks(rotation='70', fontsize = 100)
plt.xlabel('Post Date ', fontsize=100)
plt.ylabel('Post Counts', fontsize=100 )
plt.tight_layout()

# 날짜 별로 모든 Influneza 데이터에 관한 기사(Article) 갯수

In [57]:
df3 = df_infect

In [58]:
df3 = df3[df3.Influenza == "COVID-19"]

In [59]:
df3.sort_values(by='Date', ascending=True, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [60]:
f, ax = plt.subplots(figsize=(150, 100))
sns.countplot('Date', data=df3)
plt.title('COVID-19 Article Posting Counts', fontsize=100)
plt.xticks(rotation='90', fontsize = 70)
plt.yticks(rotation='70', fontsize = 100)
plt.xlabel('PostDate Values', fontsize=100)
plt.ylabel('Counts', fontsize=100 )
plt.tight_layout()

# 날짜 별로 COVID-19 에 관한 기사(Article) 갯수

여기까지 Global Infection 에 관한 Visualization ****


In [61]:
df_roam.head()
df_roam.tail()

,return,iso,arrival,departure,count
212320,20200201,za,20200123,20200131,3
212321,20200201,za,20200124,20200131,3
212322,20200201,za,20200127,20200128,3
212323,20200201,zm,20200120,20200122,3
212324,20200201,zw,20200123,20200124,3


In [62]:
df4 = df_roam

In [67]:
df4.sort_values(by='return', ascending=False)

,return,iso,arrival,departure,count
72099,20200331,jp,20200321,20200330,3
72110,20200331,jp,20200329,20200329,13
72120,20200331,kh,20200314,20200314,3
72119,20200331,kh,20200312,20200331,3
72118,20200331,kh,20200303,20200303,3
...,...,...,...,...,...
149042,20191201,gb,20191201,20191201,4
149041,20191201,gb,20191130,20191201,20
149040,20191201,gb,20191130,20191130,9
149039,20191201,gb,20191129,20191201,10


In [68]:
df4.tail()

,return,iso,arrival,departure,count
212320,20200201,za,20200123,20200131,3
212321,20200201,za,20200124,20200131,3
212322,20200201,za,20200127,20200128,3
212323,20200201,zm,20200120,20200122,3
212324,20200201,zw,20200123,20200124,3


In [70]:
df4.tail()

,return,iso,arrival,departure,count
212320,20200201,za,20200123,20200131,3
212321,20200201,za,20200124,20200131,3
212322,20200201,za,20200127,20200128,3
212323,20200201,zm,20200120,20200122,3
212324,20200201,zw,20200123,20200124,3


In [71]:
combine = pd.concat([df_roam, df_infect])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [72]:
combine

,Date,Influenza,arrival,count,departure,iso,return
0,NaT,NaN,20200130.00,3.00,20200130.00,ad,20200131.00
1,NaT,NaN,20191231.00,3.00,20200125.00,ae,20200131.00
2,NaT,NaN,20191231.00,3.00,20200131.00,ae,20200131.00
3,NaT,NaN,20200102.00,3.00,20200102.00,ae,20200131.00
4,NaT,NaN,20200104.00,3.00,20200131.00,ae,20200131.00
...,...,...,...,...,...,...,...
61717,2020-02-16,Influenza,nan,nan,nan,NaN,nan
61718,2020-02-16,Influenza,nan,nan,nan,NaN,nan
61719,2020-03-16,Influenza,nan,nan,nan,NaN,nan
61720,2020-03-16,Influenza,nan,nan,nan,NaN,nan


In [73]:
combine1 = pd.concat([df_roam, df_infect], axis=1)

In [74]:
combine1

,return,iso,arrival,departure,count,Influenza,Date
0,20200131,ad,20200130,20200130,3,Poliomyelitis,2019-12-02
1,20200131,ae,20191231,20200125,3,Acquired immuno deficiency syndrome,2019-12-02
2,20200131,ae,20191231,20200131,3,Acquired immuno deficiency syndrome,2019-12-02
3,20200131,ae,20200102,20200102,3,Acquired immuno deficiency syndrome,2019-12-03
4,20200131,ae,20200104,20200131,3,Acquired immuno deficiency syndrome,2019-12-04
...,...,...,...,...,...,...,...
212320,20200201,za,20200123,20200131,3,NaN,NaT
212321,20200201,za,20200124,20200131,3,NaN,NaT
212322,20200201,za,20200127,20200128,3,NaN,NaT
212323,20200201,zm,20200120,20200122,3,NaN,NaT


In [75]:
combine1.isnull().sum()

return            0
iso            2972
arrival           0
departure         0
count             0
Influenza    189855
Date         189855
dtype: int64

In [76]:
combine2 = pd.concat([df4, df_infect], axis=1)

In [77]:
combine2

,return,iso,arrival,departure,count,Influenza,Date
0,20200131,ad,20200130,20200130,3,Poliomyelitis,2019-12-02
1,20200131,ae,20191231,20200125,3,Acquired immuno deficiency syndrome,2019-12-02
2,20200131,ae,20191231,20200131,3,Acquired immuno deficiency syndrome,2019-12-02
3,20200131,ae,20200102,20200102,3,Acquired immuno deficiency syndrome,2019-12-03
4,20200131,ae,20200104,20200131,3,Acquired immuno deficiency syndrome,2019-12-04
...,...,...,...,...,...,...,...
212320,20200201,za,20200123,20200131,3,NaN,NaT
212321,20200201,za,20200124,20200131,3,NaN,NaT
212322,20200201,za,20200127,20200128,3,NaN,NaT
212323,20200201,zm,20200120,20200122,3,NaN,NaT
